# 1. Import modules

In [1]:
import sys
import os
import numpy as np
import gc

In [2]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from datetime import datetime

# 2. Define the device for training

In [3]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"

# 3. Define model

In [4]:
def getVGGModel():
  vgg16 = models.vgg16_bn(weights=models.vgg.VGG16_BN_Weights.IMAGENET1K_V1)

  # Fix the conv layers parameters
  for conv_param in vgg16.features.parameters():
    conv_param.require_grad = False

  # Replace w/ new classification layers
  classifications = nn.Sequential(
    nn.Linear(25088,2048),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(2048,3)
  )

  vgg16.classifier = classifications

  return vgg16

In [5]:
model = getVGGModel()
    
model.to(DEVICE)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

# 4. Define hyperparameters

In [6]:
hp = {"lr":1e-5, "beta1":0.9, "beta2":0.999, "batch_size":16, "epochs":5}

# 5. Load dataset and define data augmentation strategy

In [7]:
def load_datasets(train_path, val_path, test_path):
  val_img_transform = transforms.Compose([transforms.Resize((244,244)),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  train_img_transform = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                            transforms.RandomAffine(degrees=0, scale=(0.8,1.2), shear=0.2, translate=(0.2, 0.2)), 
                                            transforms.ToTensor(),
                                            transforms.Resize((244,244)), 
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

  train_dataset = datasets.ImageFolder(train_path, transform=train_img_transform)
  val_dataset = datasets.ImageFolder(val_path, transform=val_img_transform) 
  test_dataset = datasets.ImageFolder(test_path, transform=val_img_transform) if test_path is not None else None
  print(f"Train set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}")
  return train_dataset, val_dataset, test_dataset
    
def construct_dataloaders(train_set, val_set, test_set, batch_size, shuffle=True):
  train_dataloader = torch.utils.data.DataLoader(train_set, batch_size, shuffle)
  val_dataloader = torch.utils.data.DataLoader(val_set, batch_size) 
  test_dataloader = torch.utils.data.DataLoader(test_aset, batch_size) if test_path is not None else None
  return train_dataloader, val_dataloader, test_dataloader

In [8]:
# Please specify the path to train, cross_validation, and test images below:
train_path, val_path, test_path = "/tmp/Dataset_2/Train/", "/tmp/Dataset_2/Validation/", None
train_set, val_set, test_set = load_datasets(train_path, val_path, test_path)
train_dataloader, val_dataloader, test_dataloader = construct_dataloaders(train_set, val_set, test_set, hp["batch_size"], True)

Train set size: 1322, Validation set size: 363


# 6. Define optimizer

In [9]:
opt = torch.optim.Adam(model.parameters(),lr=hp["lr"], betas=(hp["beta1"], hp["beta2"]))

# 7. Define loss function

In [10]:
loss_fn = nn.CrossEntropyLoss()

# 8. Train model

## 8.1 Define train function

In [11]:
@torch.no_grad()
def eval_model(data_loader, model, loss_fn, DEVICE):
  model.train(False)
  model.eval()
  loss, accuracy = 0.0, 0.0
  n = len(data_loader)

  for i, data in enumerate(data_loader):
    x,y = data
    x,y = x.to(DEVICE), y.to(DEVICE)
    pred = model(x)
    loss += loss_fn(pred, y)/len(x)
    pred_label = torch.argmax(pred, axis = 1)
    accuracy += torch.sum(pred_label == y)/len(x)

  return loss/n, accuracy/n

## 8.2 Define evaluation function

In [12]:
def train(train_loader, val_loader, model, opt, loss_fn, epochs, DEVICE):
  n = len(train_loader)
  
  for epoch in range(epochs):
    model.train(True)
    
    avg_loss, avg_acc = 0.0, 0.0
    count = 0
    print(f"Epoch {epoch+1}/{epochs}:")
    start_time = datetime.now()
    for x, y in train_loader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      pred = model(x)
      loss = loss_fn(pred,y)

      opt.zero_grad()
      loss.backward()
      opt.step()

      avg_loss += loss/len(x)
      pred_label = torch.argmax(pred, axis=1)
      avg_acc += torch.sum(pred_label == y)/len(x)
      
    end_time = datetime.now()    
    print(f"Time: {(end_time-start_time).seconds}s")
    print(f"Average train loss: {avg_loss/n}, Average train accuracy: {avg_acc/n}")

    val_loss, val_acc = eval_model(val_loader, model, loss_fn, DEVICE)
    print(f"Val loss: {val_loss}, Val accuracy: {val_acc}\n")

## 8.3 Start training

In [13]:
train(train_dataloader, val_dataloader, model, opt, loss_fn, hp["epochs"], DEVICE)

Epoch 1/5:
Time: 453s
Average train loss: 0.0629410594701767, Average train accuracy: 0.49262046813964844
Val loss: 0.05285432189702988, Val accuracy: 0.655138373374939

Epoch 2/5:
Time: 450s
Average train loss: 0.04688914120197296, Average train accuracy: 0.6688252687454224
Val loss: 0.04042689874768257, Val accuracy: 0.7055335640907288

Epoch 3/5:
Time: 450s
Average train loss: 0.03946926072239876, Average train accuracy: 0.7224397659301758
Val loss: 0.037577517330646515, Val accuracy: 0.7245553135871887

Epoch 4/5:
Time: 442s
Average train loss: 0.03333243727684021, Average train accuracy: 0.7646084427833557
Val loss: 0.03764159232378006, Val accuracy: 0.7245553135871887

Epoch 5/5:
Time: 440s
Average train loss: 0.030014054849743843, Average train accuracy: 0.8033132553100586
Val loss: 0.035583458840847015, Val accuracy: 0.7366601228713989

